In [36]:
import pandas as pd
from datetime import datetime
import re
import math
import os.path

In [37]:
#Mapping Functions

def Mapping_Categ(Cat):
    rtv = Cat

    if (Cat == 'Prepare') or (Cat == 'Training'):
        rtv = 'Prepare Content for future delivry on'
    if (Cat == 'Internal MS Meeting') or (Cat == 'Front'):
        rtv = 'Meeting with'

    return rtv

def Mapping_Subject(Sub):
    rtv = 'Partners'
    rec = re.compile("^\[([a-zA-Z0-9]+)\]")

    try:
        qui = rec.findall(Sub)
        
        if len(qui) > 0:
            rtv = qui[0]
            if rtv.lower() == 'all' : rtv = 'Partners' 
    except:
        pass

    return rtv

def GenReport(Year,Month,df,out_filename):
    
    From = datetime(Year,Month,1)
    if Month < 12:
        To   = datetime(Year,Month+1,1)
    else:
        To   = datetime(Year+1,1,1)
        
    #CleanUp Columns
    df.drop(['Folder Path','Location','DisplayTo','DisplayCc','LegacyFreeBusyStatus','Importance','IsReminderSet','ReminderMinutesBeforeStart','HasAttachments','Preview','Id'],axis=1,inplace=True)

    #Rearrange Date fields
    df['Duration'] = df.End - df.Start
    df['DurationHour'] = df['Duration'].dt.total_seconds().div(3600).astype(float)
    df['DurationHour'] = df['DurationHour'].apply(lambda d: int(math.ceil(d)))
    df['week'] = df['Start'].apply(lambda d: d.isocalendar()[1])
    df['Date'] = pd.to_datetime(df['Start']).dt.normalize()
    df.set_index('Date',inplace=True)
    df = df.sort_values('Date')

    #Slice Data
    df_Z = df.loc[From:To,:].copy()
    
    #Map Activities field
    df_Z['Activity_Purpose'] = df_Z.Categories.map(Mapping_Categ)
    df_Z['Activity_Target'] = df_Z.Subject.map(Mapping_Subject)
    df_Z['Activity'] = df_Z['Activity_Purpose'] + " " + df_Z['Activity_Target']
    df_Z['Day'] = df_Z['Start'].apply(lambda d: d.strftime('%d/%m/%Y'))
    
    #Prepare before writing
    #Remove Admin and Vacances
    df_Z = df_Z.query('Activity_Purpose != "Admin"')
    df_Z = df_Z.query('Activity_Purpose != "Vacances"')
    
    #CleanUp Columns
    final_df = df_Z.drop(['Subject','Start','End','IsAllDayEvent','Categories','Activity_Purpose','Activity_Target','Duration'],axis=1)
    
    #Agregate by Week / Activity
    grp_by_week_activity = final_df.groupby(['week','Activity']).agg('sum')
    
    #Write to out filename
    if os.path.isfile(out_filename):        
        with pd.ExcelWriter(out_filename,mode='a') as writer:  
            grp_by_week_activity.to_excel(writer, sheet_name=From.strftime("%B"))
    else:
        grp_by_week_activity.to_excel(out_filename,sheet_name=f'{From.strftime("%B")} {Year}')
        
    return final_df

In [27]:
YEAR_OF_QUERY=2020
MONTH_OF_QUERY=10
OUT_EXCEL_NAME='Activité-Q2.xlsx'
IMPUT_RAW='Raw-Data-Calendar.xlsx'

In [28]:
for m in [10,11,12]:
    df_raw = pd.read_excel(IMPUT_RAW) 
    GenReport(YEAR_OF_QUERY,m,df_raw,OUT_EXCEL_NAME)

In [29]:
YEAR_OF_QUERY=2021
OUT_EXCEL_NAME='Activité-Q3.xlsx'
IMPUT_RAW='Raw-Data-Calendar.xlsx'

In [35]:
#Query Janvier
MONTH_OF_QUERY=1
df_raw = pd.read_excel(IMPUT_RAW) 
GenReport(YEAR_OF_QUERY,MONTH_OF_QUERY,df_raw,OUT_EXCEL_NAME)

,DurationHour,week,Activity,Day
Date,,,,
2021-01-04,1,1,Prepare Content for future delivry on CHROMEOS,04/01/2021
2021-01-04,2,1,Meeting with FIDUCIAL,04/01/2021
2021-01-05,3,1,Prepare Content for future delivry on Partners,05/01/2021
2021-01-05,2,1,Meeting with PM,05/01/2021
2021-01-05,1,1,Meeting with CORP,05/01/2021
2021-01-06,3,1,Prepare Content for future delivry on Partners,06/01/2021
2021-01-06,3,1,Prepare Content for future delivry on CHROMEOS,06/01/2021
2021-01-06,1,1,Meeting with PDM,06/01/2021
2021-01-07,1,1,Meeting with PDM,07/01/2021
